## The `time` Module – Advanced Exercises

This notebook contains a series of **advanced (but not too advanced)** exercises focusing on the `time` module and related tools from the standard library.

For each exercise you will see:
- A **problem description** (markdown cell)
- A **solution** (code cell), written with clean, best-practice Python

You can hide solutions or re-implement them yourself for practice.

---
### Exercise 1 – A Reusable Timing Context Manager (`perf_counter`)

**Goal:** Create a small, reusable utility to time arbitrary blocks of code using `time.perf_counter`.

**Requirements:**
1. Implement a context manager `time_block(label: str)` that:
   - Uses `perf_counter()` at `__enter__` and `__exit__`.
   - Prints the label and elapsed time in milliseconds (ms, rounded to 3 decimal places).
2. Use this context manager to compare the performance of two ways of computing the sum of squares from 0 to `n-1`:
   - A `for`-loop accumulation
   - A list comprehension combined with `sum`
3. Use `n = 2_000_00` (two hundred thousand) or a size that runs reasonably fast on your system.

This exercise practices:
- Using `perf_counter`
- Writing a clean context manager
- Simple benchmarking patterns

In [1]:
from time import perf_counter
from contextlib import contextmanager
from typing import Iterator, Callable

@contextmanager
def time_block(label: str) -> Iterator[None]:
    """Context manager that measures execution time using perf_counter."""
    start = perf_counter()
    try:
        yield
    finally:
        end = perf_counter()
        elapsed_ms = (end - start) * 1000
        print(f"{label}: {elapsed_ms:.3f} ms")

def sum_of_squares_loop(n: int) -> int:
    """Return sum of i*i for i in range(n) using a for-loop."""
    total = 0
    for i in range(n):
        total += i * i
    return total

def sum_of_squares_comprehension(n: int) -> int:
    """Return sum of i*i for i in range(n) using a generator comprehension."""
    return sum(i * i for i in range(n))

def benchmark(func: Callable[[int], int], n: int) -> int:
    """Benchmark a function taking an int and returning an int. Returns the result for verification."""
    with time_block(func.__name__):
        return func(n)

if __name__ == "__main__":
    N = 200_000

    result_loop = benchmark(sum_of_squares_loop, N)
    result_comp = benchmark(sum_of_squares_comprehension, N)

    assert result_loop == result_comp, "Results differ between implementations!"
    print("Results are equal:", result_loop)

sum_of_squares_loop: 13.464 ms
sum_of_squares_comprehension: 25.458 ms
Results are equal: 2666646666700000


---
### Exercise 2 – Retry with Exponential Backoff (`sleep` + `perf_counter`)

**Goal:** Implement a function that retries a flaky operation, waiting increasingly longer between attempts.

**Requirements:**
1. Write a function `retry_with_backoff` with this signature:
   ```python
   def retry_with_backoff(
       func: Callable[[], T],
       max_attempts: int = 5,
       base_delay: float = 0.2,
   ) -> T:
   ```
   where `T` is a type variable.
2. The function should:
   - Call `func()`.
   - If `func()` raises an exception, wait for `base_delay * 2**attempt_index` seconds and retry, up to `max_attempts`.
   - If all attempts fail, re-raise the last exception.
3. Use `time.sleep` for waiting.
4. Demonstrate the function by simulating a flaky operation that fails a few times before succeeding (you can use `random` for this).

This exercise practices:
- Combining `sleep` with control flow
- Writing robust retry logic
- Using type hints with callables

In [2]:
from time import sleep
from typing import Callable, TypeVar
import random

T = TypeVar("T")

def retry_with_backoff(
    func: Callable[[], T],
    max_attempts: int = 5,
    base_delay: float = 0.2,
) -> T:
    """Retry `func` with exponential backoff."""
    if max_attempts < 1:
        raise ValueError("max_attempts must be >= 1")

    last_exception: Exception | None = None

    for attempt in range(max_attempts):
        try:
            return func()
        except Exception as exc:
            last_exception = exc
            if attempt == max_attempts - 1:
                break
            delay = base_delay * (2 ** attempt)
            print(f"Attempt {attempt+1} failed: {exc}. Retrying in {delay:.2f}s...")
            sleep(delay)

    assert last_exception is not None
    raise last_exception

def flaky_operation_factory(fail_times: int) -> Callable[[], str]:
    """Return a function that fails `fail_times` times before succeeding."""
    remaining_failures = {"count": fail_times}

    def op() -> str:
        if remaining_failures["count"] > 0:
            remaining_failures["count"] -= 1
            raise RuntimeError("Simulated transient error")
        return "Success!"

    return op

if __name__ == "__main__":
    flaky = flaky_operation_factory(fail_times=3)
    result = retry_with_backoff(flaky, max_attempts=5, base_delay=0.1)
    print("Final result:", result)

Attempt 1 failed: Simulated transient error. Retrying in 0.10s...
Attempt 2 failed: Simulated transient error. Retrying in 0.20s...
Attempt 3 failed: Simulated transient error. Retrying in 0.40s...
Final result: Success!


---
### Exercise 3 – Seconds Until a Future UTC Time (`time`, `gmtime`)

**Goal:** Compute how many seconds remain until a given future UTC time represented as a `time.struct_time`.

**Requirements:**
1. Implement a function:
   ```python
   def seconds_until(target_utc: time.struct_time) -> float:
       ...
   ```
   that returns (possibly negative) seconds from **now (UTC)** until `target_utc`.
2. Use:
   - `time.time()` to get the current epoch seconds.
   - `calendar.timegm` to convert the `time.struct_time` to epoch seconds.
3. Demonstrate the function by creating:
   - A target time 10 seconds in the future.
   - A target time 10 seconds in the past.

This exercise practices:
- Converting between `struct_time` and epoch seconds
- Working consistently in UTC
- Simple date/time arithmetic

In [3]:
import time
from calendar import timegm
from time import gmtime

def seconds_until(target_utc: time.struct_time) -> float:
    """Return seconds from now (UTC) until `target_utc`."""
    now_epoch = time.time()
    target_epoch = timegm(target_utc)
    return target_epoch - now_epoch

if __name__ == "__main__":
    now_epoch = time.time()

    future_epoch = now_epoch + 10
    future_utc = gmtime(future_epoch)

    past_epoch = now_epoch - 10
    past_utc = gmtime(past_epoch)

    print("Seconds until (future):", seconds_until(future_utc))
    print("Seconds until (past):  ", seconds_until(past_utc))

Seconds until (future): 9.380290031433105
Seconds until (past):   -10.620011568069458


---
### Exercise 4 – Parsing ISO 8601-like Logs (`strptime`, `timegm`)

**Goal:** Parse a list of log timestamps, convert them to epoch seconds, and compute statistics.

**Scenario:** You have a list of timestamps in a simplified ISO 8601 format: `YYYY-MM-DDTHH:MM:SSZ` (UTC).

**Requirements:**
1. Given a list like:
   ```python
   logs = [
       "2023-01-01T12:00:00Z",
       "2023-01-01T12:00:05Z",
       "2023-01-01T12:01:10Z",
       "2023-01-01T12:10:00Z",
   ]
   ```
2. Implement a function `parse_iso_utc(ts: str) -> time.struct_time` using `time.strptime` with an appropriate format string.
3. Implement a function `to_epoch(ts: str) -> int` that uses `parse_iso_utc` and `calendar.timegm`.
4. Compute and print:
   - The minimum timestamp (epoch seconds).
   - The maximum timestamp.
   - The total duration between min and max in seconds.

This exercise practices:
- Parsing string timestamps
- Converting to epoch seconds
- Simple aggregation over time-series data

In [4]:
from time import strptime
from calendar import timegm
import time as _time

def parse_iso_utc(ts: str) -> time.struct_time:
    """Parse a simplified ISO 8601 UTC timestamp."""
    return strptime(ts, "%Y-%m-%dT%H:%M:%SZ")

def to_epoch(ts: str) -> int:
    """Convert an ISO 8601-like UTC timestamp to epoch seconds (int)."""
    st = parse_iso_utc(ts)
    return timegm(st)

if __name__ == "__main__":
    logs = [
        "2023-01-01T12:00:00Z",
        "2023-01-01T12:00:05Z",
        "2023-01-01T12:01:10Z",
        "2023-01-01T12:10:00Z",
    ]

    epochs = [to_epoch(ts) for ts in logs]

    min_ts = min(epochs)
    max_ts = max(epochs)
    duration = max_ts - min_ts

    print("Epoch timestamps:", epochs)
    print("Earliest:", min_ts)
    print("Latest:  ", max_ts)
    print("Duration (s):", duration)

Epoch timestamps: [1672574400, 1672574405, 1672574470, 1672575000]
Earliest: 1672574400
Latest:   1672575000
Duration (s): 600


---
### Exercise 5 – Formatting Epoch Times as UTC Strings (`strftime`, `gmtime`)

**Goal:** Write a helper function to format epoch times into a standard UTC string.

**Requirements:**
1. Implement:
   ```python
   def format_utc(epoch: float) -> str:
       """Return 'YYYY-MM-DD HH:MM:SS UTC' formatted string."""```
2. Internally:
   - Use `time.gmtime` to convert from epoch seconds to `struct_time`.
   - Use `time.strftime` with an appropriate format string.
3. Demonstrate it on:
   - The current time (`time.time()`).
   - A fixed epoch such as `0` (the Unix epoch).
   - A future time like `time.time() + 3600` (one hour later).

This exercise practices:
- Formatting timestamps into human-readable UTC strings
- Using `strftime` effectively with custom formats

In [5]:
from time import gmtime, strftime, time as time_now

def format_utc(epoch: float) -> str:
    """Return a string `YYYY-MM-DD HH:MM:SS UTC` for the given epoch seconds."""
    st = gmtime(epoch)
    return strftime("%Y-%m-d %H:%M:%S UTC", st)

if __name__ == "__main__":
    now = time_now()
    one_hour_later = now + 3600

    print("Now:           ", format_utc(now))
    print("Unix epoch:    ", format_utc(0))
    print("One hour later:", format_utc(one_hour_later))

Now:            2025-11-d 15:35:29 UTC
Unix epoch:     1970-01-d 00:00:00 UTC
One hour later: 2025-11-d 16:35:29 UTC
